# Task one

### Stage 1:

Import the input dataset, convert the data types, then save as a
parquet file.
- schema.json file should be used to load the dataset
- The output parquet file must be called: films

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [2]:
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName('imdb-munging') \
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 13:50:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/23 13:50:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/23 13:50:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
sc.setLogLevel("WARN")

In [4]:
file = "name.basics.tsv.gz"
path = r'../datasets/%s' % file

# Define a Schema
schema = 'nconst STRING, primaryName STRING, birthYear DATE, deathYear DATE, primaryProfession STRING, knownForTitles STRING'
names = spark.read.csv(path, schema=schema, sep="\t", header=True, dateFormat="yyyy")

names = names.drop('birthYear').drop('deathYear').drop('primaryProfession').drop('knownForTitles')


In [5]:
# 1.5 million move ratings
file = "title.ratings.tsv.gz"
path = r'../datasets/%s' % file

# Define a Schema
schema = 'tconst STRING not null, averageRating DECIMAL(4,2), numVotes INTEGER'
ratings = spark.read.csv(path, schema=schema, sep="\t", header=True)

In [6]:
# 10,248,884 rows
file = "title.principals.tsv.gz"
path = r'../datasets/%s' % file

# Define a Schema
schema = 'tconst STRING, ordering INTEGER, nconst STRING, category STRING, job STRING, characters STRING'
principals = spark.read.csv(path, schema=schema, sep="\t", header=True).select('tconst', 'nconst', 'ordering', 'job')


In [7]:
file = "title.basics.tsv.gz"
path = r'../datasets/%s' % file

# Define a Schema
schema = 'tconst STRING not null, titleType STRING, primaryTitle STRING, originalTitle STRING, isAdult BOOLEAN, startYear DATE, endYear DATE, runtimeMinutes INTEGER, genres STRING'
titles = spark.read.csv(path, schema=schema, sep="\t", header=True, dateFormat="yyyy").select('tconst', 'titleType', 'primaryTitle', 'startYear', 'runtimeMinutes', 'genres')


In [8]:
# Join the ratings to the titles on tconst -> 11_297_218 1_512_899 titles
titles = titles.join(ratings, on='tconst')

In [9]:
# join `pricipals` to `names` to get a list of people involved in the film
principals = principals.join(names, on='nconst')

#principals = principals.sort(['tconst', 'ordering'])

# principals is 61_278_693 rows, highest ordering value is 75, 
# then about #10_248_884 when collect_set() to an array of persons
principals.printSchema()

root
 |-- nconst: string (nullable = true)
 |-- tconst: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- primaryName: string (nullable = true)



In [10]:
#principals.groupBy('job').count().sort(f.desc('count')).show(50, truncate=False)
#principals.groupBy('job').avg('ordering').show(50, truncate=False)
#principals.groupBy('job', 'ordering').count().sort(f.desc('count')).show(50, truncate=False)
#principals.filter((principals.job == 'director') & (principals.ordering <= 3)).show(50)

In [11]:
# 'job' column is unreliable
#principals = principals.filter(principals.job.isin('\\N','director', 'producer', 'writer', 'composer', 'creator'))\
#            .groupby('tconst').agg( f.collect_set('primaryName').alias('persons') )

In [12]:
# perform a reverse explode, do not sort_array() of persons
# at this point, principals becomes a key-val table of tconst:[persons]
principals = principals.groupby('tconst').agg( f.collect_set('primaryName').alias('persons') )
principals.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- persons: array (nullable = false)
 |    |-- element: string (containsNull = false)



In [13]:
# filter the number of films down to something more managable
titles = titles.filter( (titles['titleType'] == 'movie')
                        & (titles['genres'] != '\\N')
                        & (titles['averageRating'] >= 6) 
                        & (ratings['numVotes'] >= 500)
                        & (titles['runtimeMinutes'].isNotNull())
                      ).drop('titleType')

# reduced to about 28170 movies

In [14]:
# make the genres column an array of geners for each title
titles = titles.withColumn("genres", f.split(titles['genres'], r'\s*,\s*'))

In [15]:
# add the principals (persons) array for each film
titles = titles.join(principals, on='tconst')

In [16]:
# Sanitise the column names
col_names = {'tconst': 'film_id', 
            'primaryTitle': 'title', 
            'startYear': 'year', 
            'runtimeMinutes': 'duration', 
            'primaryName': 'person',
            'averageRating': 'rating', 
            'numVotes': 'vote_count'}
titles = titles.withColumnsRenamed(col_names)

In [17]:
# strip the leading "tt" from the film_id
titles = titles.withColumn('film_id', f.replace( titles.film_id, f.lit('tt'), f.lit('') ).cast("int"))

In [18]:
titles.schema.json()

'{"fields":[{"metadata":{},"name":"film_id","nullable":true,"type":"integer"},{"metadata":{},"name":"title","nullable":true,"type":"string"},{"metadata":{},"name":"year","nullable":true,"type":"date"},{"metadata":{},"name":"duration","nullable":true,"type":"integer"},{"metadata":{},"name":"genres","nullable":true,"type":{"containsNull":false,"elementType":"string","type":"array"}},{"metadata":{},"name":"rating","nullable":true,"type":"decimal(4,2)"},{"metadata":{},"name":"vote_count","nullable":true,"type":"integer"},{"metadata":{},"name":"persons","nullable":false,"type":{"containsNull":false,"elementType":"string","type":"array"}}],"type":"struct"}'

In [19]:
titles.printSchema()

root
 |-- film_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- year: date (nullable = true)
 |-- duration: integer (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- rating: decimal(4,2) (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- persons: array (nullable = false)
 |    |-- element: string (containsNull = false)



24/12/23 01:50:46 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [45]:
# Write the processed IMDB data to a parquet file
path = "../output/films"
titles.write.parquet(path, mode='overwrite')

In [21]:
titles.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [cast(replace(tconst#50, tt, ) as int) AS film_id#135, primaryTitle#52 AS title#126, startYear#55 AS year#124, runtimeMinutes#57 AS duration#121, genres#105, averageRating#28 AS rating#123, numVotes#29 AS vote_count#125, persons#95]
   +- SortMergeJoin [tconst#50], [tconst#33], Inner
      :- Sort [tconst#50 ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(tconst#50, 200), ENSURE_REQUIREMENTS, [plan_id=661]
      :     +- Project [tconst#50, primaryTitle#52, startYear#55, runtimeMinutes#57, split(genres#58, \s*,\s*, -1) AS genres#105, averageRating#28, numVotes#29]
      :        +- BroadcastHashJoin [tconst#50], [tconst#27], Inner, BuildRight, false
      :           :- Project [tconst#50, primaryTitle#52, startYear#55, runtimeMinutes#57, genres#58]
      :           :  +- Filter (((((isnotnull(titleType#51) AND isnotnull(genres#58)) AND (titleType#51 = movie)) AND NOT (genres#58 = \N)) AND isnotnull(ru

In [22]:
sc.stop()